In [1]:
import tensorflow as tf
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import copy
from mrcnn.config import Config
import cv2
import cv2 as cv
from keras import backend as K
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import samples.coco.coco as coco


%matplotlib inline 

# cesta pro ulozeni vysledku
MODEL_DIR = '/training_summaries/'
# cesta k natrenovanemu modelu
MODEL_PATH = "training_summaries/mask_rcnn_coralclef2020_epoch-083.h5"



Using TensorFlow backend.


In [2]:
class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 13 + 1
    DETECTION_MIN_CONFIDENCE = 0.5


config = InferenceConfig()
config.display()

class_names = ['BG','c_hard_coral_branching', 'c_hard_coral_submassive', 'c_hard_coral_boulder',
               'c_hard_coral_encrusting', 'c_hard_coral_table', 'c_hard_coral_foliose', 'c_hard_coral_mushroom',
               'c_soft_coral', 'c_soft_coral_gorgonian', 'c_sponge', 'c_sponge_barrel', 'c_fire_coral_millepora',
               'c_algae_macro_or_leaves']


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                26
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [3]:
K.clear_session()

# vytvoreni modelu v modu inference
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# nacteni natrenovaneho modelu
model.load_weights(MODEL_PATH, by_name=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [5]:
test_path = './CoralClef2020/imageCLEFcoral2020_test_v4/'
img_names = os.listdir(test_path)

In [ ]:
data = []
data_mask = []



progbar = tf.keras.utils.Progbar(len(img_names))

for imgi in range(0, len(img_names)):

    progbar.update(imgi)
    
    # nacteni obrazku
    file_names = img_names[imgi]
    image = skimage.io.imread(os.path.join(test_path, file_names))

    # detekce
    results = model.detect([image], verbose=0)
    r = results[0]

    # konverze detekovanych dat do pozadovaneho formatu
    row = convert_data(file_names, r)
    data.append(row)
    
    # model predikuje segmenty jako 2d pole, pro odevzdani jsou potreba jen hranicni body
    seg = transform_masks(r['masks'])
    
    #odstraneni prekrivajicich se bodu
    k = 0 
    for s in seg:
        
        #odstranit prkryvajici se body
        ss = copy.deepcopy(s)
        ind = []
        ii = 0
        for i in s:
            jj = 0
            for j in s:
                if(i[0][0] == j[0][0] and i[0][1] == j[0][1] and ii != jj):
                    ind.append(ii)
                jj = jj + 1
            ii = ii + 1  
        

        for i in range(len(ss)-2):
            if((s[i][0][0] == s[i+1][0][0] and s[i+1][0][0] == s[i+2][0][0]) and 
              ((s[i+1][0][1] < s[i][0][1] and s[i][0][1] < s[i+2][0][1]) or 
               (s[i+1][0][1] > s[i][0][1] and s[i][0][1] > s[i+2][0][1]) or
               (s[i+1][0][1] < s[i+2][0][1] and s[i+2][0][1] < s[i][0][1]) or 
               (s[i+1][0][1] > s[i+2][0][1] and s[i+2][0][1] > s[i][0][1]))):
                ind.append(i)
                
            if((s[i][0][1] == s[i+1][0][1] and s[i+1][0][1] == s[i+2][0][1]) and 
              ((s[i+1][0][0] < s[i][0][0] and s[i][0][0] < s[i+2][0][0]) or 
               (s[i+1][0][0] > s[i][0][0] and s[i][0][0] > s[i+2][0][0]) or
               (s[i+1][0][0] < s[i+2][0][0] and s[i+2][0][0] < s[i][0][0]) or 
               (s[i+1][0][0] > s[i+2][0][0] and s[i+2][0][0] > s[i][0][0]))):
                ind.append(i)
        
            
        
        #odstranit body z listu
        if(len(ind) > 0):
            print(ind, k)
            sss = []
            for i in range(len(ss)):
                if(i not in ind):
                    sss.append(list(ss[i]))

            seg[k] = copy.deepcopy(np.array(sss))
        k = k + 1
        
    
    # konverze dat do pozadovaneho formatu
    row = convert_mask_data(file_names, r, seg)
    data_mask.append(row)
    
    

In [121]:
with open('test_MRCNN_mask.txt','w') as f:
    f.write('\n'.join(data_mask))
    
with open('test_MRCNN.txt','w') as f:
    f.write('\n'.join(data))

In [6]:
def transform_masks(masks):
    x = np.zeros((3024, 4032,3), np.dtype('uint8'))
    x[:,:,0] = x[:,:,0] + 125
    x[:,:,1] = x[:,:,1] + 125
    x[:,:,2] = x[:,:,2] + 125
    seg = []

    for k in range(0, masks.shape[2]):
        im = np.zeros((3024, 4032,3), np.dtype('uint8'))

        im[:,:,0] = copy.deepcopy(masks[:,:,k])*255
        im[:,:,1] = copy.deepcopy(masks[:,:,k])*255
        im[:,:,2] = copy.deepcopy(masks[:,:,k])*255

        imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
        ret, thresh = cv.threshold(imgray, 127, 255, 0)
        contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

        m = 0
        mi = 0
        for i in range(len(contours)):
            if len(contours[i]) > m:
                mi = i
                m = len(contours[i])
                cnt = contours[i]


        lt = 1
        old_cnt = copy.deepcopy(cnt)
        new_cnt = copy.deepcopy(cnt)
        while(old_cnt.shape[0] > 150):
            new_cnt = []
            lt = lt + 1
            for i in range(0, len(old_cnt)-1, 2):
                l1 = old_cnt[i]
                l2 = old_cnt[i+1]
                l = np.sqrt((l2[0][0] - l1[0][0])**2 + (l2[0][1] - l1[0][1])**2)
                if(l < lt):
                    new_cnt.append(l1)
                else:
                    new_cnt.append(l1)
                    new_cnt.append(l2)
            new_cnt = np.array(new_cnt)
            old_cnt = copy.deepcopy(new_cnt)


        f = []
        f.append(copy.deepcopy(new_cnt))
        seg.append(copy.deepcopy(new_cnt))

    return seg

In [7]:
def convert_mask_data(filename, r, seg):
    bbox_data = {}
    
    for j in range(0, len(seg)):
        s = seg[j]
        b = ''
        k = 1
        class_id = r['class_ids'][j]
        class_name = class_names[int(class_id)]
        confidence = round(r['scores'][j],2)
        for c in s:
            if(k < len(s)):
                b = b + '{}+{}+'.format(c[0][0], c[0][1])
                k = k + 1
            else:
                b = b + '{}+{}'.format(c[0][0], c[0][1])
 
        if(class_name not in bbox_data.keys()):
            bbox_data.update({class_name:[[confidence,b]]})
        else:
            bbox_data[class_name].append([confidence,b])
            

    row = filename.split('.')[0] + ';'
    k = 1
    for c in bbox_data.keys():
        row = row + c + ' '
        i = 1
        for b in bbox_data[c]:
            if(i < len(bbox_data[c])):
                row = row + '{:.2f}:{},'.format(b[0], b[1])
                i = i + 1
            else:
                row = row + '{:.2f}:{}'.format(b[0], b[1])
        if(k < len(bbox_data.keys())):
            row = row + ';'
            k = k + 1
    return row

In [8]:
def convert_data(filename, r):
    y_pred_resized = r['rois']
    bbox_data = {}
    for i in range(0, len(y_pred_resized)):
        box = y_pred_resized[i]
        class_id = r['class_ids'][i]
        class_name = class_names[int(class_id)]
        confidence = round(r['scores'][i],2)
        ymin = int(box[0])
        xmin = int(box[1])
        ymax = int(box[2])
        xmax = int(box[3])
        if(xmin < 0):
            xmin = 0
        if(ymin < 0):
            ymin = 0
        if(ymax > 3024):
            ymax = 3024
        if(xmax > 4032):
            xmax = 4032
        width = xmax - xmin
        height = ymax - ymin


        if(class_name not in bbox_data.keys()):
            bbox_data.update({class_name:[[confidence,width,height,xmin,ymin]]})
        else:
            bbox_data[class_name].append([confidence,width,height,xmin,ymin])

    row = filename.split('.')[0] + ';'
    k = 1
    for c in bbox_data.keys():
        row = row + c + ' '
        i = 1
        for b in bbox_data[c]:
            if(i < len(bbox_data[c])):
                row = row + '{:.2f}:{}x{}+{}+{},'.format(b[0], b[1], b[2], b[3], b[4])
                i = i + 1
            else:
                row = row + '{:.2f}:{}x{}+{}+{}'.format(b[0], b[1], b[2], b[3], b[4])
        if(k < len(bbox_data.keys())):
            row = row + ';'
            k = k + 1
    return row